In [ ]:
# SETUP: workaround to avoid ipynb import errors
import os
import sys

candidate_roots = [
    os.getcwd(),
    os.path.dirname(os.getcwd()),
    os.path.dirname(os.path.dirname(os.getcwd())),
]
for root in candidate_roots:
    src_dir = os.path.join(root, "src")
    if os.path.isdir(src_dir) and src_dir not in sys.path:
        sys.path.append(src_dir)
        break

In [ ]:
from workload_configuration import analysis_config, get_graph_and_node_to_remove
from application.node_removal_graph_analyser import get_node_removal_impact
import pandas as pd
import time

# method to run the analysis
def run_analysis_from_config(config):
    # get run parameters
    gtype = config["graph_type"]
    size = config["size"]
    metric_name = config["metric_name"]
    metric_func = config["metric_func"]

    try:
        # generate the graph
        G, node_to_remove = get_graph_and_node_to_remove(gtype, size)
        # get time and impact on nodes (discarded) for the analysis
        time, _ = get_node_removal_impact(G, [node_to_remove], metric_func)

        # generate report
        result = {
            "graph": gtype,
            "size": size,
            "metric": metric_name,
            "time": time,
            #"impact": impact
        }
        
        print(result)
        return result
    except Exception as e:
        # catch any errors that might happen
        result = {
            "graph": gtype,
            "size": size,
            "metric": metric_name,
            "error": str(e)
        }

        print(result)
        return result


# create a dictionary from the results from the runs from the config
results = [run_analysis_from_config(cfg) for cfg in analysis_config]

# save results to csv
results_df = pd.DataFrame(results)
results_df.to_csv(f"../../results/performance-analysis/big-graphs/{time.time()}.csv")